In [1]:
import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix, hstack
from keras.utils import np_utils
import pickle
from sklearn.linear_model import Ridge
from tqdm import tqdm
from sklearn.model_selection import KFold

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [15]:
svd_dir = '../feature/svd_5.pkl'
tifi_dir = '../feature/tifi_min4.pkl'
pro_output = '../10fold_pro/10foldprob_ridge_baseline.csv'
output = '../output/prob_ridge_baseline2.csv'
modify_tifi = False
modefi_svd = False

In [3]:
t1=time.time()
train = pd.read_csv('../input/new_data/train_set.csv')
test = pd.read_csv('../input/new_data/test_set.csv')
test_id = pd.read_csv('../input/new_data/test_set.csv')[["id"]].copy()

In [4]:
y=(train["class"]-1).astype(int)

/usr/local/lib/python3.5/dist-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [5]:
all_data = pd.concat([train, test])
all_data = all_data.reset_index(drop=True)
train_offset = train.shape[0]

In [6]:
if modefi_svd:
    all_text_svd = []
    column="article"
    vec = TfidfVectorizer(ngram_range=(1,2),min_df=4, max_df=0.9,use_idf=1,smooth_idf=1, sublinear_tf=1)
    full_article = vec.fit_transform(all_data[column])
    test_article = vec.transform(test[column])
    train_article = vec.transform(train[column])

    ### SVD Components ###
    n_comp = 5
    svd_obj = TruncatedSVD(n_components=n_comp, algorithm='arpack')
    svd_obj.fit(full_article)
    train_svd = pd.DataFrame(svd_obj.transform(train_article))
    test_svd = pd.DataFrame(svd_obj.transform(test_article))
    train_svd.columns = ['svd_params_'+str(i+1) for i in range(n_comp)]
    test_svd.columns = ['svd_params_'+str(i+1) for i in range(n_comp)]
    all_text_svd.extend(train_svd.columns)
    train = pd.concat([train, train_svd], axis=1)
    test = pd.concat([test, test_svd], axis=1)
    del full_article, train_article, test_article
    name = ['svd_params_'+str(i+1) for i in range(n_comp)]
    train_svd = csr_matrix(pd.get_dummies(train[name],
                                          sparse=True).values)
    test_svd = csr_matrix(pd.get_dummies(test[name],
                                          sparse=True).values)
    with open(svd_dir,'wb') as fout:
        pickle.dump([train_svd,test_svd],fout)
else:
    train_svd, test_svd= pickle.load(open(svd_dir,'rb'))

In [7]:
if modify_tifi:
    column="word_seg"
    vec = TfidfVectorizer(ngram_range=(1,2),min_df=4, max_df=0.9,use_idf=1,smooth_idf=1, sublinear_tf=1)
    trn_term_doc = vec.fit_transform(train[column])
    test_term_doc = vec.transform(test[column])
    with open(tifi_dir,'wb') as fout:
        pickle.dump([trn_term_doc,test_term_doc],fout)
else:
     trn_term_doc, test_term_doc= pickle.load(open(tifi_dir,'rb'))

In [8]:
X_train = hstack((trn_term_doc, train_svd)).tocsr()
X_test = hstack((test_term_doc, test_svd)).tocsr()

In [9]:
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, y, test_size=0.15, random_state=42)

In [10]:
name = ["class_prob_%s"%i for i in range(1,20)]
one_y_train=np_utils.to_categorical(y,num_classes=20)
# one_y_val=np_utils.to_categorical(Y_valid,num_classes=20)
test_prob = pd.DataFrame([],columns=["class_prob_%s"%i for i in range(1,20)])
train_prob = pd.DataFrame([],columns=["class_prob_%s"%i for i in range(1,20)])

In [11]:
from sklearn.linear_model import Ridge
from tqdm import tqdm
for i in tqdm(range(20)):
    model = Ridge(solver="sag", fit_intercept=True, random_state=205)
    model.fit(X_train, one_y_train[:,i])
    test_prob['class_prob_%s'%(i+1)] = model.predict(X_test)
    train_prob['class_prob_%s'%(i+1)] = model.predict(X_train)

# val_result = np.argmax(test_prob[name].values,axis=1)
# print ('score is {}'.format(f1_score(Y_valid, val_result, average='macro')))

100%|██████████| 20/20 [21:50<00:00, 65.52s/it]


In [16]:
# test_prob=pd.DataFrame(test_pred)
# test_prob.columns=["class_prob_%s"%i for i in range(1,20)]
test_prob["id"]=list(test_id["id"])
test_prob.to_csv(pro_output,index=None)

In [17]:
preds=np.argmax(test_prob[name].values,axis=1)
test_pred=pd.DataFrame(preds)
test_pred.columns=["class"]
test_pred["class"]=(test_pred["class"]+1).astype(int)
print(test_pred.shape)
print(test_id.shape)
test_pred["id"]=list(test_id["id"])
test_pred[["id","class"]].to_csv(output,index=None)
t2=time.time()
print("time use:",t2-t1)

(102277, 1)
(102277, 1)
time use: 1486.0870008468628


In [14]:
with open('../feature/stackint_pkl_file/ridge.pkl','wb') as fout:
    pickle.dump([train_prob,test_prob],fout)

In [18]:
a,b = pickle.load(open('../feature/stackint_pkl_file/lr.pkl','rb'))

FileNotFoundError: [Errno 2] No such file or directory: '../feature/stackint_pkl_file/lr.pkl'